# Graph Balancing Algorithm Explanation

This notebook provides a step-by-step walkthrough of the 1.75-approximation algorithm for Graph Balancing.


## Problem Definition

Given a weighted multigraph, we want to orient edges to minimize the maximum vertex load.

Load of vertex v = dedicated_load(v) + sum of weights of edges oriented towards v


In [1]:
import sys
sys.path.append('..')

from core import Graph, Orientation
from algorithm import lp_balance
from lp_solver import solve_lp3
from rounding import round_procedure


## Example 1: Simple Instance

Let's start with a simple example to understand the algorithm.


In [2]:
# create a simple graph
vertices = [0, 1, 2]
edges = [(0, 1), (1, 2)]
edge_weights = {0: 0.4, 1: 0.3}
dedicated_loads = {0: 0.1, 1: 0.2, 2: 0.1}

graph = Graph(vertices, edges, edge_weights, dedicated_loads)

print(f"Graph: {len(graph.vertices)} vertices, {len(graph.edges)} edges")
print(f"Edge weights: {graph.edge_weights}")
print(f"Dedicated loads: {graph.dedicated_loads}")


Graph: 3 vertices, 2 edges
Edge weights: {0: 0.4, 1: 0.3}
Dedicated loads: {0: 0.1, 1: 0.2, 2: 0.1}


### Step 1: Solve LP3

First, we solve the linear program LP3 to get a fractional solution.


In [3]:
# solve LP3
x = solve_lp3(graph)

print("LP3 Solution:")
for edge_idx, (u, v) in enumerate(graph.edges):
    x_eu = x.get((edge_idx, u), 0.0)
    x_ev = x.get((edge_idx, v), 0.0)
    print(f"  Edge {edge_idx} ({u}-{v}): x_{edge_idx}{u} = {x_eu:.4f}, x_{edge_idx}{v} = {x_ev:.4f}")
    print(f"    Sum: {x_eu + x_ev:.4f} (should be 1.0)")

# check load constraints
print("\nLoad constraints:")
for v in graph.vertices:
    load = graph.get_dedicated_load(v)
    for edge_idx in graph.get_incident_edges(v):
        u, w = graph.edges[edge_idx]
        if v == u:
            load += x.get((edge_idx, u), 0.0) * graph.get_edge_weight(edge_idx)
        else:
            load += x.get((edge_idx, w), 0.0) * graph.get_edge_weight(edge_idx)
    print(f"  Vertex {v}: {load:.4f} (should be <= 1.0)")


LP3 Solution:
  Edge 0 (0-1): x_00 = 0.5552, x_01 = 0.4448
    Sum: 1.0000 (should be 1.0)
  Edge 1 (1-2): x_11 = 0.3137, x_12 = 0.6863
    Sum: 1.0000 (should be 1.0)

Load constraints:
  Vertex 0: 0.3221 (should be <= 1.0)
  Vertex 1: 0.4720 (should be <= 1.0)
  Vertex 2: 0.3059 (should be <= 1.0)


### Step 2: Round the Solution

Now we round the fractional solution to get an integral orientation.


In [4]:
# round the solution
orientation_mapping = round_procedure(graph, x)

print("Rounded Orientation:")
for edge_idx, target in orientation_mapping.items():
    u, v = graph.edges[edge_idx]
    print(f"  Edge {edge_idx} ({u}-{v}) -> Vertex {target}")


Rounded Orientation:
  Edge 0 (0-1) -> Vertex 1
  Edge 1 (1-2) -> Vertex 2


### Step 3: Compute Final Loads

Let's see the final loads on each vertex.


In [5]:
# create orientation object
orientation = Orientation(graph, orientation_mapping)

print("Final Loads:")
for v in graph.vertices:
    load = orientation.compute_load(v)
    print(f"  Vertex {v}: {load:.4f}")

makespan = orientation.compute_makespan()
print(f"\nMakespan: {makespan:.4f}")
print(f"Within 1.75 guarantee: {makespan <= 1.75}")


Final Loads:
  Vertex 0: 0.1000
  Vertex 1: 0.6000
  Vertex 2: 0.4000

Makespan: 0.6000
Within 1.75 guarantee: True


## Summary

The algorithm works in three main steps:

1. **Solve LP3**: Get a fractional solution that satisfies all constraints
2. **Round**: Convert fractional solution to integral orientation
3. **Verify**: Check that makespan <= 1.75 * optimal

The key insight is that the rounding procedure maintains the approximation guarantee by carefully handling:
- Big edges (p_e > 0.5): at most one per vertex
- Leaf assignments: when x_eu * p_e <= 0.75
- Tree assignments: for trees of big edges
- Rotations: to eliminate fractional values
